<a href="https://colab.research.google.com/github/TarkanAl-Kazily/awbw_replay_parser/blob/main/AWBW_Replay_Parser_and_Kantbeis_Replay_Archive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running the AWBW Replay Parser toolset on Kantbei's Replay Archive

This notebook runs the AWBW Replay Parser to open and load replays from Kantbei's Replay Archive. It's primarily configured to load all the replays in a batch to check for parsing errors or other errors, but it can be used as a base for a strategy analysis on the high tier games in the archive.

See the main project on Github here: [https://github.com/TarkanAl-Kazily/awbw_replay_parser](https://github.com/TarkanAl-Kazily/awbw_replay_parser)

Access Kantbei's dataset from Google Drive here: [https://drive.google.com/drive/folders/1ziWc1mVm9lOydIquCG8EsTb3aYvpdaPp?usp=sharing](https://drive.google.com/drive/folders/1ziWc1mVm9lOydIquCG8EsTb3aYvpdaPp?usp=sharing)

In [ ]:
# One time setup for this notebook
import os

os.environ['REPO'] = "https://github.com/TarkanAl-Kazily/awbw_replay_parser.git"
os.environ['BRANCH'] = "main"

# Clone the awbw_replay_parser repository
!cd /content/
!rm -rf awbw_replay_parser
!git clone --branch $BRANCH $REPO
!pip install -r awbw_replay_parser/requirements.txt

# Install the awbw_replay module into our runtime
!cd awbw_replay_parser && python setup.py install

To use Kantbei's dataset, you need to add a shortcut to the directory to your drive through Google Drive. Then from the Google Colab Files sidebar you can mount your Google Drive, and the data should be accessible under `drive/MyDrive/Kantbei's AWBW Replay Archive`.

In [ ]:
from logging import exception
import os
import glob
from awbw_replay.replay import AWBWReplay
from awbw_replay.awbw import AWBWGameState, AWBWGameAction

KANTBEI_REPLAY_DIR = "/content/drive/MyDrive/Kantbei's AWBW Replay Archive"

# Glob all the replay archive files. Replace this list with the specific files you'd like to load.
replay_list = glob.glob(os.path.join(KANTBEI_REPLAY_DIR, "*.zip"))

replay_stats = {}

def attempt_parsing_replay(filename):
  """
  Tries to parse a given replay file. Returns a dictionary summarizing the result.
  
  Handles all exceptions, making it safe to continue running afterwards.
  """
  result = {
    "filename": filename,
    "actions": None, # Count how many actions are in the file
    "turns": None, # Count how many turns are in the file
    "states": None, # Count how many states we could iterate through
    "players": None, # Count the number of players in the file
    "exception": None, # The exception string in the case of failure
    "exception_obj": None, # The actual exception object
    "success": False, # If we reached the end of parsing all states
  }
  try:
    with AWBWReplay(filename) as replay:
      actions = list(replay.actions())
      result["actions"] = len(actions)
      result["turns"] = len(replay.turns())
      result["states"] = 0
      state = AWBWGameState(replay_initial=replay.game_info())
      result["states"] += 1
      result["players"] = len(state.players)
      for action in actions:
        state = state.apply_action(AWBWGameAction(action))
        result["states"] += 1

      result["success"] = True
  except Exception as e:
    result["exception"] = str(e)
    result["exception_obj"] = e
    print(f"Got exception: {e}")
  
  return result

for filename in replay_list:
  if filename in replay_stats and replay_stats[filename]["success"]:
    print(f"Skipping already successful file {filename}")
    continue
  print(f"Testing file {filename}...")
  replay_stats[filename] = attempt_parsing_replay(filename)

successful_replays = [stat["filename"] for stat in replay_stats.values() if stat["success"]]

print(f"Successfully parsed states from {len(successful_replays)} files (out of {len(replay_list)}).")

In [3]:
error_list = [result for result in replay_stats.values() if not result["success"]]

for result in error_list:
  print(f"{result['filename']} - {result['exception']} at state {result['states']}")

/content/drive/MyDrive/Kantbei's AWBW Replay Archive/039 League (Std) p0.32451 (Kindle) vs SonjaTheSuperior (Kindle) - SeaShoal Ocarina.zip - 83532217 at state 309
/content/drive/MyDrive/Kantbei's AWBW Replay Archive/069 League (Fog) shoeLv3 (Hawke) vs Hevol (Hawke) - To Athos.zip - Not a gzipped file (b'PK') at state None
/content/drive/MyDrive/Kantbei's AWBW Replay Archive/135 League (Fog) RiceMuncher (Rachel) vs Kevin or hell (Kindle) - Stolen Seconds.zip - 'nextPId' at state 1524
/content/drive/MyDrive/Kantbei's AWBW Replay Archive/145 League (Std) Djandjan (Eagle) vs starflash250 (Olaf) - From Wharf to Wilds.zip -  at state 61
/content/drive/MyDrive/Kantbei's AWBW Replay Archive/196 League (Fog) Chinese Name (Sonja) vs eagle. (Sami) - Mors Celeri.zip - Error -3 while decompressing data: invalid distance too far back at state None
/content/drive/MyDrive/Kantbei's AWBW Replay Archive/214 League (Std) Vechi (Grimm) vs the FANG (Adder) - Dubious Procession.zip -  at state 73
/content/

In [4]:
for result in error_list:
  print(f"\"{result['filename']}\",")

"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/039 League (Std) p0.32451 (Kindle) vs SonjaTheSuperior (Kindle) - SeaShoal Ocarina.zip",
"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/069 League (Fog) shoeLv3 (Hawke) vs Hevol (Hawke) - To Athos.zip",
"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/135 League (Fog) RiceMuncher (Rachel) vs Kevin or hell (Kindle) - Stolen Seconds.zip",
"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/145 League (Std) Djandjan (Eagle) vs starflash250 (Olaf) - From Wharf to Wilds.zip",
"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/196 League (Fog) Chinese Name (Sonja) vs eagle. (Sami) - Mors Celeri.zip",
"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/214 League (Std) Vechi (Grimm) vs the FANG (Adder) - Dubious Procession.zip",
"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/234 League (Std) Vechi (Grimm) vs the FANG (Adder) - Dubious Procession.zip",
"/content/drive/MyDrive/Kantbei's AWBW Replay Archive/258 Custo